https://stackoverflow.com/questions/103512/why-use-static-castintx-instead-of-intx  
https://stackoverflow.com/questions/28002/regular-cast-vs-static-cast-vs-dynamic-cast

http://www.cplusplus.com/doc/tutorial/typecasting/

Standard Conversion: when smaller datatype is promoted to bigger datatype  
Gaurenteed to get same exact value  

# Implicit Casting

In [ ]:
// implicit conversion of classes:
#include <iostream>
using namespace std;

class A {};

class B {
public:
  // conversion from A (constructor):
  B (const A& x) {}
  // conversion from A (assignment):
  B& operator= (const A& x) {return *this;}
  // conversion to A (type-cast operator)
  // destination type followed by paranthesis
  operator A() {return A();}
};

int main ()
{
  A foo;
  B bar = foo;    // calls constructor
  bar = foo;      // calls assignment
  foo = bar;      // calls type-cast operator
  return 0;
}

# Explicit Casting

Used to make *single paramter constructors* explicit so object is not implicitly casted from another type

https://weblogs.asp.net/kennykerr/Explicit-Constructors

In [ ]:
class Array
{
public:
    Array(size_t count);
    // etc.
};


Array array = 123; // but this is wrong because it allows for: 
void UseArray(const Array& array);
UseArray(123);
// --------------------Where this avoids the issue-------------------------
class Array
{
public:
    explicit Array(size_t size);
    // etc.
};

https://stackoverflow.com/questions/121162/what-does-the-explicit-keyword-mean

In [ ]:
// explicit:
#include <iostream>
using namespace std;

class A {};

class B {
public:
  explicit B (const A& x) {}
  B& operator= (const A& x) {return *this;}
  operator A() {return A();}
};

void fn (B x) {}

int main ()
{
  A foo;
  B bar (foo);
  bar = foo;
  foo = bar;
  
//  fn (foo);  // not allowed for explicit ctor.
  fn (bar);  

  return 0;
}

# Type-Casting

### In order to prevent casting like this:

In [ ]:
// class type-casting
#include <iostream>
using namespace std;

class Dummy {
    double i,j;
};

class Addition {
    int x,y;
  public:
    Addition (int a, int b) { x=a; y=b; }
    int result() { return x+y;}
};

int main () {
  Dummy d;
  Addition * padd;
  padd = (Addition*) &d;
  cout << padd->result();
  return 0;
}

- dynamic_cast <new_type> (expression)
- reinterpret_cast <new_type> (expression)
- static_cast <new_type> (expression)
- const_cast <new_type> (expression)

replacing:
- (new_type) expresssion
- new_type(expresssion)

# dynamic_cast

Only used for pointers and references to classes: polymorphic down and upcast  
Has runtime overhead

In [1]:
// dynamic_cast
#include <iostream>
#include <exception>
using namespace std;

class Base { virtual void dummy() {} };
class Derived: public Base { int a; };

In [2]:
{
    try{
        Base* derived_obj = new Derived;
        Base* base_obj = new Base;
        Derived * obj;

        obj = dynamic_cast<Derived*>(derived_obj);
        if (obj==nullptr) // nullptr or 0
            cout << "Null pointer on first type-cast.\n";

        obj = dynamic_cast<Derived*>(base_obj);
        if (obj==nullptr) 
            cout << "Null pointer on second type-cast.\n";

    } catch (exception& e) {
        cout << "Exception: " << e.what();
    }
}

Null pointer on second type-cast.


# Static Casts

Used for ordinary casts, only compile time (no runtime checks)

In [3]:
{
    Base * a = new Base;
    Derived * b = static_cast<Derived*>(a);
}

- Convert from void* to any pointer type. In this case, it guarantees that if the void* value was obtained by converting from that same pointer type, the resulting pointer value is the same.
- Convert integers, floating-point values and enum types to enum types.


- Explicitly call a single-argument constructor or a conversion operator.
- Convert to rvalue references.
- Convert enum class values into integers or floating-point values.
- Convert any type to void, evaluating and discarding the value.

### The rule is:

- Use static_cast when you know that the target type is valid.
- Use dynamic_cast when you're not sure, and you need the program to look up the object's runtime type for you.

# Object Slicing

When using casts, remember to use pointers, otherwise object slicing will occur. 

https://stackoverflow.com/a/36370071  
https://stackoverflow.com/a/274636

In [4]:
#include <string>
#include <vector>
#include <iostream>
using namespace std;
class A {
public:
  virtual const string f() const { return "A"; }
};
class B : public A {
public:
  const string f() const { return "B"; }
};

In [5]:
{
    vector<A> v;
    v.push_back(B());
    cout << v.at(0).f() << endl;
}

A


Method expects an A object (not a pointer), so it slices off anything that is not an A. To prevent this, use pointers:

In [6]:
{
    vector<A*> v;
    v.push_back(new B());
    cout << v.at(0)->f() << endl;
}

B


**`static_cast<Type>(object)`** -> copy constructor is called, new object is created (pretend as if C-style cast was used, it would create a new object)  
**`static_cast<Type*>(&object)`** -> one good boi

# reinterpret_cast

Binary **copy** of one type to another  

The only guarantee is that a pointer cast to an integer type (for ex.) large enough to fully contain it (such as intptr_t), is guaranteed to be able to be cast back to a valid pointer.

# const_cast

In [ ]:
// const_cast
#include <iostream>
using namespace std;

void print (char * str)
{
  cout << str << '\n';
}

int main () {
  const char * c = "sample text";
  print ( const_cast<char *> (c) );
  return 0;
}

Can remove or set const  
Can remove const from non-const value and be modified, if it was originally const, then modifying will result in undefined behavior

# typeid

`#include <typeinfo>`

- i: integer
- Pi: pointer to integer variable
- Ppi: pointer to a pointer to integer variable

In [2]:
#include <iostream>
#include <typeinfo>

{
    // typeid
    using namespace std;

    int* a,b;
    a=0; b=0;
    if (typeid(a) != typeid(b))
    {
        cout << "a and b are of different types:\n";
        cout << "a is: " << typeid(a).name() << '\n';
        cout << "b is: " << typeid(b).name() << '\n';
    }
}

a and b are of different types:
a is: Pi
b is: i


In [3]:
// typeid, polymorphic class
class Base { virtual void f(){} };
class Derived : public Base {};

In [4]:
#include <iostream>
#include <typeinfo>
#include <exception>

{
using namespace std;

  try {
    Base* a = new Base;
    Base* b = new Derived;
    cout << "a is: " << typeid(a).name() << '\n';
    cout << "b is: " << typeid(b).name() << '\n';
    cout << "*a is: " << typeid(*a).name() << '\n';
    cout << "*b is: " << typeid(*b).name() << '\n';
  } catch (exception& e) { cout << "Exception: " << e.what() << '\n'; }
  return 0;
}

a is: P4Base
b is: P4Base
*a is: 4Base
*b is: 7Derived


0